<p style="text-align: center; font-size: 192%"> Computational Finance </p>
<img src="../img/ABSlogo.svg" alt="LOGO" style="display:block; margin-left: auto; margin-right: auto; width: 90%;">
<p style="text-align: center; font-size: 150%"> Week 4: Asset Pricing </p>
<p style="text-align: center; font-size: 75%"> <a href="#copyrightslide">Copyright</a> </p>

In [38]:
#silence some warnings
import warnings
warnings.filterwarnings('ignore')

# Outline

* More pandas: Hierarchical Indexing
* Merging databases

# More pandas: Hierarchical Indexing

* The MultiIndex object ([user guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)) is part of `pandas`.

* It is an index with multiple levels (row index or column headers).

* This allows you to display higher dimensional data into a lower dimension.

* Flexible: tools for reshaping and aggregation.

*The example is from Data Science Methods  course by Cees Diks and Bram Wouters.*

## Creating a MultiIndex

* When creating a DataFrame, you can create a MultiIndex by using nested lists (for the `index` or `columns`).

**Example:** A dataFrame with row and column indices of MultiIndex type.

Number of ECTS obtained per year, semester by different students and programs.

In [59]:
import numpy as np
import pandas as pd

frame = pd.DataFrame(
    np.array([24, 24, 6, 24, 24, 6, 30, 24, 6, 18, 12, 6, 24, 18, 6, 24, 18, 12]).reshape(6,3),
    index=[['17-18', '17-18', '18-19', '18-19', '19-20', '19-20'], ['I','II','I', 'II', 'I', 'II']],
    columns=[['Robert', 'Esther', 'Esther'],['Finance', 'Finance', 'Ectrics']])

frame.index.names = ['year', 'semester']
frame.columns.names = ['name', 'program']

frame

name            Robert  Esther        
program        Finance Finance Ectrics
year  semester                        
17-18 I             24      24       6
      II            24      24       6
18-19 I             30      24       6
      II            18      12       6
19-20 I             24      18       6
      II            24      18      12

In [90]:
frame.index

MultiIndex([('17-18',  'I'),
            ('17-18', 'II'),
            ('18-19',  'I'),
            ('18-19', 'II'),
            ('19-20',  'I'),
            ('19-20', 'II')],
           names=['year', 'semester'])

In [41]:
type(frame.index)

pandas.core.indexes.multi.MultiIndex

* The module `MultiIndex` offers methods `from_arrays()` and `from_tuples()` to create a MultiIndex:

In [42]:
pd.MultiIndex.from_arrays([['17-18', '17-18', '18-19', '18-19', '19-20', '19-20'], 
                           ['I','II','I', 'II', 'I', 'II']], names = ['year','semester'])

MultiIndex([('17-18',  'I'),
            ('17-18', 'II'),
            ('18-19',  'I'),
            ('18-19', 'II'),
            ('19-20',  'I'),
            ('19-20', 'II')],
           names=['year', 'semester'])

* It can be more convenient to use `from_product()` when all combinations of the elements in each level are included:

In [43]:
pd.MultiIndex.from_product([['17-18', '18-19', '19-20'], ['I','II']], names = ['year','semester'])

MultiIndex([('17-18',  'I'),
            ('17-18', 'II'),
            ('18-19',  'I'),
            ('18-19', 'II'),
            ('19-20',  'I'),
            ('19-20', 'II')],
           names=['year', 'semester'])

### Indexing and slicing
* Selection works similar to a DataFrame without MultiIndex. 
* Select the number of ECTS that Esther obtained in the Finance program.

In [92]:
frame['Esther', 'Ectrics'] # or frame['Esther']['Finance']

year   semester
17-18  I            6
       II           6
18-19  I            6
       II           6
19-20  I            6
       II          12
Name: (Esther, Ectrics), dtype: int32

* The method `xs()` can be used to slice rows or columns (default is rows). It takes a level argument, for easy selection at any level.

In [95]:
frame.xs(('18-19', 'I'), level=(0,1)) # xs takes level argument (and optional axis argument)

name            Robert  Esther        
program        Finance Finance Ectrics
year  semester                        
18-19 I             30      24       6

### Swap levels

Swapping the levels of the row MultiIndex.

In [46]:
frame

name            Robert  Esther        
program        Finance Finance Ectrics
year  semester                        
17-18 I             24      24       6
      II            24      24       6
18-19 I             30      24       6
      II            18      12       6
19-20 I             24      18       6
      II            24      18      12

In [69]:
# Year/ semester 바꿈
frame.swaplevel(axis=1)  # more general: reorder_levels([1, 0], axis=1) where you can give a permutation of the levels

program        Finance        Ectrics
name            Robert Esther  Esther
year  semester                       
17-18 I             24     24       6
      II            24     24       6
18-19 I             30     24       6
      II            18     12       6
19-20 I             24     18       6
      II            24     18      12

In [97]:
frame.sort_index(axis = 1)

name            Esther          Robert
program        Ectrics Finance Finance
year  semester                        
17-18 I              6      24      24
      II             6      24      24
18-19 I              6      24      30
      II             6      12      18
19-20 I              6      18      24
      II            12      18      24

In [98]:
frame.sort_index(axis = 0)

name            Robert  Esther        
program        Finance Finance Ectrics
year  semester                        
17-18 I             24      24       6
      II            24      24       6
18-19 I             30      24       6
      II            18      12       6
19-20 I             24      18       6
      II            24      18      12

* May want to sort after this, using `sort_index(level, axis)` where you can specify the level at which you want to sort.

### Reshaping

Using `stack` and `unstack` ([user guide](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#reshaping-by-stacking-and-unstacking)) to turn the row index 'year' into a column index and the column index 'program' into a row index.

In [48]:
frame

name            Robert  Esther        
program        Finance Finance Ectrics
year  semester                        
17-18 I             24      24       6
      II            24      24       6
18-19 I             30      24       6
      II            18      12       6
19-20 I             24      18       6
      II            24      18      12

In [104]:
frame.unstack(level='year').stack(level='program')

name             Esther             Robert            
year              17-18 18-19 19-20  17-18 18-19 19-20
semester program                                      
I        Ectrics      6     6     6    NaN   NaN   NaN
         Finance     24    24    18   24.0  30.0  24.0
II       Ectrics      6     6    12    NaN   NaN   NaN
         Finance     24    12    18   24.0  18.0  24.0

In [81]:
frame.unstack(level='semester').stack(level='semester') 

name            Esther          Robert
program        Ectrics Finance Finance
year  semester                        
17-18 I              6      24      24
      II             6      24      24
18-19 I              6      24      30
      II             6      12      18
19-20 I              6      18      24
      II            12      18      24

In [49]:
frame.unstack(level='year').stack(level='program')  # Adds NaN if field is empty

name             Esther             Robert            
year              17-18 18-19 19-20  17-18 18-19 19-20
semester program                                      
I        Ectrics      6     6     6    NaN   NaN   NaN
         Finance     24    24    18   24.0  30.0  24.0
II       Ectrics      6     6    12    NaN   NaN   NaN
         Finance     24    12    18   24.0  18.0  24.0

In [84]:
frame

name            Robert  Esther        
program        Finance Finance Ectrics
year  semester                        
17-18 I             24      24       6
      II            24      24       6
18-19 I             30      24       6
      II            18      12       6
19-20 I             24      18       6
      II            24      18      12

### Aggregation

* Aggregation at particular level using `ufuncs` (sum, mean, etc.) is easy.

* Calculating the maximum of `frame` for each pair of 'year' and 'name'.

In [86]:
# 선택한 것 중에서 가장 큰 값을 선택
frame.max(level='year').max(level='name', axis=1)

name,Robert,Esther
year,,
17-18,24,24
18-19,30,24
19-20,24,18


* Alternative in a DataFrame *without* MultiIndex is to use [`groupby()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html), but more verbose.

## Merging databases

*This section uses parts of Tomislav Ladika's Python bootcamp from the UvA course Data Analytics (MIF).*



* Information not always stored in a single database $\rightarrow$ need to merge data sets.

* Suppose that we want to combine data from CRSP and Compustat.
    * Here we use a sample (previously downloaded).
* Download and inspect the data sets before merging.

* Are there common variables (columns) that we can use to merge?

* Let's read the Compustat data...

In [52]:
import pandas as pd
## Open all data frames
compustat_data = pd.read_csv("./data/sample_data_compustat.txt", sep="\t")   # Can read any text file; takes comma as default separator.

## Create a new 'date' column in Compustat, which is the 'datadate' column in YYYYMM format
compustat_data['datadate'] = compustat_data['datadate'].astype(str)  # Ensure that it is a string
compustat_data['date'] = compustat_data['datadate'].str[0:6]  
compustat_data['date'] = compustat_data['date'].astype(int) 

compustat_data.head(10)

,gvkey,company_name,year,datadate,stock_price,assets,liabilities,sales,earnings,net_income,dividends,capex,date
0,1487,AMERICAN INTERNATIONAL GROUP,2000,20001231,98.5625,306577.0,264146.0,45972.0,11340.0,5636.0,335.0,4465.0,200012
1,1487,AMERICAN INTERNATIONAL GROUP,2001,20011231,79.4000,492982.0,437121.0,62402.0,14834.0,5363.0,743.0,5115.0,200112
2,1487,AMERICAN INTERNATIONAL GROUP,2002,20021231,57.8500,561229.0,498393.0,67482.0,11841.0,5519.0,467.0,6226.0,200212
3,1487,AMERICAN INTERNATIONAL GROUP,2003,20031231,66.2800,678346.0,603590.0,81303.0,18452.0,9274.0,584.0,6640.0,200312
4,1487,AMERICAN INTERNATIONAL GROUP,2004,20041231,65.6700,801145.0,716442.0,96831.0,18440.0,9839.0,755.0,5810.0,200412
5,1487,AMERICAN INTERNATIONAL GROUP,2005,20051231,68.2300,853370.0,761743.0,108340.0,21918.0,10477.0,1615.0,8211.0,200512
6,1487,AMERICAN INTERNATIONAL GROUP,2006,20061231,71.6600,979414.0,869768.0,113489.0,28932.0,14048.0,1690.0,7106.0,200612
7,1487,AMERICAN INTERNATIONAL GROUP,2007,20071231,58.3000,1060505.0,954182.0,110250.0,18959.0,6200.0,1964.0,5642.0,200712
8,1487,AMERICAN INTERNATIONAL GROUP,2008,20081231,1.5700,860418.0,797692.0,8062.0,-80053.0,-99289.0,1105.0,4817.0,200812
9,1487,AMERICAN INTERNATIONAL GROUP,2009,20091231,29.9800,847585.0,748550.0,97260.0,6327.0,-10949.0,0.0,12101.0,200912


* ... and the CRSP data.

In [53]:
crsp_data = pd.read_csv("./data/sample_data_crsp.txt", sep="\t")

## Similarly, modify the 'date' column in CRSP
crsp_data['date'] = crsp_data['date'].astype(str)
crsp_data['date'] = crsp_data['date'].str[0:6]  
crsp_data['date'] = crsp_data['date'].astype(int)

crsp_data.head(10)

,permno,permco,company_name,date,ticker,cusip,stock_price,volume,bid,ask,total_shares
0,13407,54084,FACEBOOK INC,201205,FB,30303M10,29.6000,11917422,29.62,29.62,633492
1,13407,54084,FACEBOOK INC,201206,FB,30303M10,31.0950,6674417,31.10,31.11,641000
2,13407,54084,FACEBOOK INC,201207,FB,30303M10,21.7100,5188160,21.68,21.69,674605
3,13407,54084,FACEBOOK INC,201208,FB,30303M10,18.0580,11488052,18.05,18.06,674605
4,13407,54084,FACEBOOK INC,201209,FB,30303M10,21.6600,10545365,21.65,21.66,949000
5,13407,54084,FACEBOOK INC,201210,FB,30303M10,21.1100,10995323,21.11,21.12,949000
6,13407,54084,FACEBOOK INC,201211,FB,30303M10,28.0000,15063199,27.97,27.98,949000
7,13407,54084,FACEBOOK INC,201212,FB,30303M10,26.6197,11858717,26.62,26.63,1671000
8,13407,54084,FACEBOOK INC,201301,FB,30303M10,30.9810,16750772,30.97,30.98,1684185
9,13407,54084,FACEBOOK INC,201302,FB,30303M10,27.2500,9561095,27.24,27.25,1684185


### Difference between data sources

* Merging is only possible if the DataFrames have *exactly* the same values. 
* Compare:
    * `'AMERICAN INTERNATIONAL GROUP'` of `'company_name'` in Compustat
    * `'AMERICAN INTERNATIONAL GROUP INC'` of `'company_name'` in CRSP
* Compare:        
    * `'datadate'` value of `'20001231'` in Compustat
    * `'date'` value of `'20001229'` in CRSP
    * We need monthly data, but CRSP uses the last trading day of each month, while Compustat uses the last calendar day.

### How to proceed?
1. Find some common information that is in both data frames to be merged. 
    - For example, both Compustat and CRSP contain company names (and though not shown, both also have stock tickers and CUSIP numbers)
2. Match the common values in data frame 1 to those of data frame 2
    - For example, match 'AMERICAN INTERNATIONAL GROUP' in Compustat to 'AMERICAN INTERNATIONAL GROUP INC' in CRSP
    - Unfortunately, this often must be done by hand. Smart computer algorithms or regular expressions can help.
3. Create a file that lists each of the matching values across data frames
4. A key challenge: Company names, tickers, etc. can change over time! Each data vendor has its own policy for updating these identifiers.
    - Compustat only lists the most recent company name for all dates, while CRSP lists the company name as of each date
    - Google changed name to Alphabet Inc. in a 2015 reorganization, but in Compustat financials going back to 2004 are listed under 'ALPHABET INC'    

* To highlight the merge procedure, first open a matching file (previously created).

In [54]:
compustat_crsp_linkfile = pd.read_csv("./data/sample_compustat_crsp_linkfile.txt", sep="\t")
compustat_crsp_linkfile.head(10)

,gvkey,date,permco
0,1690,200001,7
1,1690,200002,7
2,1690,200003,7
3,1690,200004,7
4,1690,200005,7
5,1690,200006,7
6,1690,200007,7
7,1690,200008,7
8,1690,200009,7
9,1690,200010,7


* We will implement the merge in Pandas using a two-step process. 
* First, merge the Compustat data frame with the link file data frame, to add the 'permco' identifier to Compustat.

In [87]:
## Merge the data frames by 'gvkey' and 'date'
merged_data = pd.merge(compustat_data, compustat_crsp_linkfile, on=['gvkey', 'date'], how='inner',suffixes= ['_data','_crsp'])   

merged_data.head()

,gvkey,company_name,year,datadate,stock_price,assets,liabilities,sales,earnings,net_income,dividends,capex,date,permco
0,1487,AMERICAN INTERNATIONAL GROUP,2000,20001231,98.5625,306577.0,264146.0,45972.0,11340.0,5636.0,335.0,4465.0,200012,137
1,1487,AMERICAN INTERNATIONAL GROUP,2001,20011231,79.4000,492982.0,437121.0,62402.0,14834.0,5363.0,743.0,5115.0,200112,137
2,1487,AMERICAN INTERNATIONAL GROUP,2002,20021231,57.8500,561229.0,498393.0,67482.0,11841.0,5519.0,467.0,6226.0,200212,137
3,1487,AMERICAN INTERNATIONAL GROUP,2003,20031231,66.2800,678346.0,603590.0,81303.0,18452.0,9274.0,584.0,6640.0,200312,137
4,1487,AMERICAN INTERNATIONAL GROUP,2004,20041231,65.6700,801145.0,716442.0,96831.0,18440.0,9839.0,755.0,5810.0,200412,137


* Then merge the modified Compustat data frame (which includes the 'permco' identifier) to the CRSP data frame.
* Note that duplicate column names that are not merged on (e.g. 'company_name'), are renamed by pandas.

In [56]:
## Merge with CRSP by 'permco' and 'date'
merged_data = pd.merge(merged_data, crsp_data, on=['permco', 'date'], how='inner')
merged_data.head()

,gvkey,company_name_x,year,datadate,stock_price_x,assets,liabilities,sales,earnings,net_income,...,permco,permno,company_name_y,ticker,cusip,stock_price_y,volume,bid,ask,total_shares
0,1487,AMERICAN INTERNATIONAL GROUP,2000,20001231,98.5625,306577.0,264146.0,45972.0,11340.0,5636.0,...,137,66800,AMERICAN INTERNATIONAL GROUP INC,AIG,02687478,98.5625,933908,98.0625,99.0625,2315555
1,1487,AMERICAN INTERNATIONAL GROUP,2001,20011231,79.4000,492982.0,437121.0,62402.0,14834.0,5363.0,...,137,66800,AMERICAN INTERNATIONAL GROUP INC,AIG,02687478,79.4000,869410,79.0200,79.4000,2612479
2,1487,AMERICAN INTERNATIONAL GROUP,2002,20021231,57.8500,561229.0,498393.0,67482.0,11841.0,5519.0,...,137,66800,AMERICAN INTERNATIONAL GROUP INC,AIG,02687478,57.8500,1111171,57.5300,57.6900,2608595
3,1487,AMERICAN INTERNATIONAL GROUP,2003,20031231,66.2800,678346.0,603590.0,81303.0,18452.0,9274.0,...,137,66800,AMERICAN INTERNATIONAL GROUP INC,AIG,02687478,66.2800,1149087,66.1000,66.1400,2607946
4,1487,AMERICAN INTERNATIONAL GROUP,2004,20041231,65.6700,801145.0,716442.0,96831.0,18440.0,9839.0,...,137,66800,AMERICAN INTERNATIONAL GROUP INC,AIG,02687478,65.6700,1226366,65.7100,65.7400,2604571


### Important decisions when merging

* Which observations to keep?
    * Data merging = data cleaning.
    * Coverage and frequency may differ: 
        * Compustat contains all firms that file financial statements in the United States.
        * CRSP contains stock prices of all firms that trade on U.S. stock exchanges.
    * Depends on ultimate goal of analysis.
* What variables to merge on?
    * Study what variables uniquely define observations!
        * E.g. company name AND date, not just company.
    * Use those in the merge command.
    * Otherwise, duplicate values.

## Summary

* **MultiIndex** is an index with multiple levels. It allows to store multidimensional data in two- or one- dimensional object.
* **Merging data** is necessary when not all data is in the same database.
* Match identifiers from both databases using a link file.

<section id="copyrightslide">

# Copyright Statement
* Course slides were created by Simon Broda for Python 2.7 $-$ Andreas Rapp adapted them to Python 3.6. 
* Week 4 slides were created by Bart Keijsers. The hierarchical indexing example is from the UvA course Data Science Methods by Cees Diks and Bram Wouters. The merging databases are from Tomislav Ladika's Python bootcamp for the UvA course Data Analytics (MIF).
* This work is licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/).
* All figures have been produced for this course using Python. Empirical results are based on public data available from [FRED](https://fred.stlouisfed.org/), [Quandl/WIKI](https://www.quandl.com/databases/WIKIP), and [Yahoo Finance](https://finance.yahoo.com/).
* More information on Simon Broda's [Github](https://github.com/s-broda/ComputationalFinance/blob/master/LICENSE.md).